# Reconocimiento de imágenes con YOLO9000
**Universidad de Chile**<br>
**Facultad de Ciencias, Física y Matemáticas**<br>
**Departamento de Ciencias de Computación**<br>
**CC5509 - Reconocimiento de Patrones**<br>
<br>
*Profesores:*<br>
**José M. Saavedra R.**<br>
**Mauricio Cerda Villablanca**<br>
<br>
*Ayudante:*<br>
**Camila Álvarez I.**<br>
<br>
*Alumnos:*<br>
**Cristobal Dotte**<br>
**Gabriel De La Parra**<br>

## Introducción

En este proyecto, se trabajó con [YOLO9000](https://pjreddie.com/darknet/yolo/) para hacer detección y clasificación de objetos en imágenes.

El [paper](https://arxiv.org/abs/1612.08242) fue escrito por Joseph Redmon y Ali Farhadi en Diciembre del 2016, por lo que se puede considerar como un trabajo en el *state-of-the-art*.

YOLO (You Only Look Once) en la versión YOLO9000 propone varias mejoras al sistema YOLO original de los mismos autores. Esta versión lleva el nombre 9000 ya que puede detectar sobre 9000 clases de objetos. Las mejores a esta versión provienen de sugerencias de otros autores y técnicas implementadas por otros sistemas. 

Uno de los aspectos más importantes de YOLO, es que puede correr a 67FPS con un mAP (*Mean Average Precision*) del 76.8 en VOC 2007. Este resultado es mucho mejor que otras técnicas existentes actualmente y a una velocidad mucho mayor. Los detalles sobre la comparación se pueden encontrar en el Paper.

El objetivo de este proyecto será realizar detección sobre carteles de precios en imágenes de frutas y verduras de ferias libres y la vega.

## Procedimiento

Se realizó el siguiente procedimiento para lograr el reconocimiento y clasificación de las imágenes:

1. Construcción del dataset
    1. Obtención de imágenes
    2. Preprocesamiento
    2. Etiquetado
    3. Adaptación de etiquetas
2. Entrenamiento
    1. Descarga YOLO
    2. Compilación
    3. Pruebas
    4. Archivos de configuración
    4. Train y training set
    5. Entrenamiento
3. Pruebas
    1. Pruebas
    2. Resultados
4. Conclusiones

## Dataset

En este proyecto, se trabajará con un dataset generado desde cero para este proyecto. El dataset está compuesto por 878 imágenes. Las fotos fueron etiquetadas ocupando la herramienta [LabelImg](https://github.com/tzutalin/labelImg). Dicha herramienta tiene un formato de salida que no es el mismo que requiere YOLO. Fue necesario crear un script para convertir los archivos de salida.

### Obtención de imágenes

Las imágenes fueron tomadas en la vega y en dos ferias libres de Santiago de Chile.

A continuación se presenta un sample de algunas de estas fotos.

In [20]:
from IPython.display import Image, HTML, display
from glob import glob
imagesList=''.join( ["<img style='width: 300px; margin: 2px; float: left; border: 1px solid black;' src='%s' />" % str(s) 
                 for s in sorted(glob('feria labeled/*.jpg')[1:100:10]) ])
display(HTML(imagesList))

## Preprocesamiento

Cada imagen tiene originalmente nombres como: `Foto 03-11-17 14 47 04.jpg` y tiene una resolución de `3264x2448`

### Resize
Si bien la resolución inicial tiene más detalle, no es necesario tanta resolución para entrenar las imagenes, debido a que YOLO hace transformación de las imagenes. Otro problema de trabajar con las imagenes a tan gran resolución, es que copiarlas al servidor ocupa mucho más recursos/tiempo. El set original tiene 2.5gb. El nuevo, 90 mb. 

Como se verá más adelante, se entrenó con la resolución de `418x418`. Se redujo en tamaño al `20%`, que corresponde a `693x490`.

``` bash
$ for a in *.jpg; do convert "$a" -resize 20% resized/"$a"; done
```

![resize](images/resize.png)

### Rename
Los nombres de las imagenes no influyen mucho, sin embargo son maś complicados para seleccionar al momento de usarlos para testear. Por lo mismo, se renombraron las imagenes de forma secuencial. El código que se utilizó es el siguiente.

``` bash
$ ls | cat -n | while read n f; do mv "$f" "$n.jpg"; done
```

![rename](images/rename.png)

## Etiquetado

Se utilizó la herramienta [LabelImg](https://github.com/tzutalin/labelImg) para etiquetar las imagenes. A continuación se presenta una muestra de la interfáz gráfica de la herramienta:

![labelimg](images/labelimg.png)

Uno de los beneficios de utilizar esta herramienta, es que se pueden utilizar distintas clases para las imágenes. Adicionalmente, la herramienta tiene comandos:
- `w`: crear un cuadro
- `a`: imagen anterior
- `d`: imagen siguiente

Por cada imagen, se crea un archivo de formato xml. El archivo tiene el siguiente contenido:

In [27]:
!head -25 1.xml

<annotation>
	<folder>feria</folder>
	<filename>1.jpg</filename>
	<path>/home/clgadel/labeler/feria/1.jpg</path>
	<source>
		<database>Unknown</database>
	</source>
	<size>
		<width>653</width>
		<height>490</height>
		<depth>3</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>manzana</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>192</xmin>
			<ymin>285</ymin>
			<xmax>227</xmax>
			<ymax>317</ymax>
		</bndbox>
	</object>
